### Bài 1:

In [1]:
import sqlite3
import math

# Kết nối và đăng ký hàm SQRT
conn = sqlite3.connect('samplee.db')
conn.create_function("SQRT", 1, lambda x: math.sqrt(x) if x is not None and x >= 0 else None)
cursor = conn.cursor()

# Tạo lại bảng (nếu cần)
cursor.execute('DROP TABLE IF EXISTS my_table')
cursor.execute('''
CREATE TABLE my_table (
    A REAL,
    B REAL
)
''')

# Chèn dữ liệu
sample_data = [(2, 4), (3, 6), (5, 8), (7, 10), (9, 14)]
cursor.executemany('INSERT INTO my_table (A, B) VALUES (?, ?)', sample_data)
conn.commit()

# Truy vấn tính hệ số tương quan
query = '''
SELECT 
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) *
     SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B))) AS r_AB
FROM my_table
'''

result = cursor.execute(query).fetchone()
print("Hệ số tương quan Pearson r_AB:", result[0])

conn.close()


Hệ số tương quan Pearson r_AB: 0.9894382924043013


### Bài 2:

In [2]:
import sqlite3
import pandas as pd

# MỞ KẾT NỐI
conn = sqlite3.connect('car_scores.db')
cursor = conn.cursor()

try:
    # Tạo bảng nếu chưa tồn tại
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS car_scores (
        Day VARCHAR(10),
        Car VARCHAR(1),
        Score FLOAT
    );
    ''')

    # Chèn dữ liệu
    cursor.executemany('''
    INSERT INTO car_scores (Day, Car, Score) VALUES (?, ?, ?)
    ''', [
        ('Day 1', 'A', 8),
        ('Day 1', 'B', 9),
        ('Day 1', 'C', 7),
        ('Day 2', 'A', 7.5),
        ('Day 2', 'B', 8.5),
        ('Day 2', 'C', 7),
        ('Day 3', 'A', 6),
        ('Day 3', 'B', 7),
        ('Day 3', 'C', 8),
        ('Day 4', 'A', 7),
        ('Day 4', 'B', 6),
        ('Day 4', 'C', 5)
    ])

    # Commit thay đổi
    conn.commit()

    # Truy vấn và in kết quả
    df = pd.read_sql_query('SELECT * FROM car_scores', conn)
    print(df)

finally:
    # ĐÓNG KẾT NỐI
    cursor.close()
    conn.close()


      Day Car  Score
0   Day 1   A    8.0
1   Day 1   B    9.0
2   Day 1   C    7.0
3   Day 2   A    7.5
4   Day 2   B    8.5
5   Day 2   C    7.0
6   Day 3   A    6.0
7   Day 3   B    7.0
8   Day 3   C    8.0
9   Day 4   A    7.0
10  Day 4   B    6.0
11  Day 4   C    5.0


In [4]:
# Kết nối database SQLite
conn = sqlite3.connect('car_scores.db')
cursor = conn.cursor()

# Thực hiện câu lệnh SELECT và tạo thêm cột Category
df_category = pd.read_sql_query('''
SELECT 
    Day,
    Car,
    Score,
    CASE 
        WHEN Score <= 6 THEN 'Low'
        WHEN Score <= 8 THEN 'Medium'
        ELSE 'High'
    END AS Category
FROM car_scores;
''', conn)
print("Dữ liệu sau khi thêm cột Category:")
print(df_category)

# Bảng tần suất của Car và Category 
df_car_freq = pd.read_sql_query('''
SELECT 
    Car,
    Category,
    COUNT(*) AS Frequency
FROM (
    SELECT 
        Car,
        CASE 
            WHEN Score <= 6 THEN 'Low'
            WHEN Score <= 8 THEN 'Medium'
            ELSE 'High'
        END AS Category
    FROM car_scores
) AS sub
GROUP BY Car, Category
ORDER BY Car, Category;
''', conn)
print("\nBảng tần suất Car và Category :")
print(df_car_freq)


Dữ liệu sau khi thêm cột Category:
      Day Car  Score Category
0   Day 1   A    8.0   Medium
1   Day 1   B    9.0     High
2   Day 1   C    7.0   Medium
3   Day 2   A    7.5   Medium
4   Day 2   B    8.5     High
5   Day 2   C    7.0   Medium
6   Day 3   A    6.0      Low
7   Day 3   B    7.0   Medium
8   Day 3   C    8.0   Medium
9   Day 4   A    7.0   Medium
10  Day 4   B    6.0      Low
11  Day 4   C    5.0      Low

Bảng tần suất Car và Category :
  Car Category  Frequency
0   A      Low          1
1   A   Medium          3
2   B     High          2
3   B      Low          1
4   B   Medium          1
5   C      Low          1
6   C   Medium          3


In [5]:
# Bảng tần suất của Day và Category 
df_day_freq = pd.read_sql_query('''
SELECT 
    Day,
    CASE 
        WHEN Score <= 6 THEN 'Low'
        WHEN Score <= 8 THEN 'Medium'
        ELSE 'High'
    END AS Category,
    COUNT(*) AS Frequency
FROM car_scores
GROUP BY Day, Category
ORDER BY Day, Category;
''', conn)
print("\n Bảng tần suất Day và  Category:")
print(df_day_freq)


 Bảng tần suất Day và  Category:
     Day Category  Frequency
0  Day 1     High          1
1  Day 1   Medium          2
2  Day 2     High          1
3  Day 2   Medium          2
4  Day 3      Low          1
5  Day 3   Medium          2
6  Day 4      Low          2
7  Day 4   Medium          1


In [6]:
#Kiểm định
from scipy.stats import chi2_contingency

# Dữ liệu Car vs Category
car_data = pd.DataFrame({
    'Car': ['A', 'A', 'B', 'B', 'B', 'C', 'C'],
    'Category': ['Low', 'Medium', 'High', 'Low', 'Medium', 'Low', 'Medium'],
    'Frequency': [1, 3, 2, 1, 1, 1, 3]
})

car_table = car_data.pivot_table(index='Car', columns='Category', values='Frequency', fill_value=0)
print("Bảng tần suất (Car vs Category):\n", car_table)

# Thực hiện kiểm định Chi bình phương
chi2_car, p_car, dof_car, expected_car = chi2_contingency(car_table)
print("\nKết quả kiểm định Chi-square cho Car:")
print(f"Chi2 statistic = {chi2_car:.4f}")
print(f"p-value = {p_car:.4f}")

# Dữ liệu Day vs Category
day_data = pd.DataFrame({
    'Day': ['Day 1', 'Day 1', 'Day 2', 'Day 2', 'Day 3', 'Day 3', 'Day 4', 'Day 4'],
    'Category': ['High', 'Medium', 'High', 'Medium', 'Low', 'Medium', 'Low', 'Medium'],
    'Frequency': [1, 2, 1, 2, 1, 2, 2, 1]
})

day_table = day_data.pivot_table(index='Day', columns='Category', values='Frequency', fill_value=0)
print("\nBảng tần suất (Day vs Category):\n", day_table)

# Thực hiện kiểm định Chi bình phương
chi2_day, p_day, dof_day, expected_day = chi2_contingency(day_table)
print("\nKết quả kiểm định Chi-square cho Day:")
print(f"Chi2 statistic = {chi2_day:.4f}")
print(f"p-value = {p_day:.4f}")

Bảng tần suất (Car vs Category):
 Category  High  Low  Medium
Car                        
A            0    1       3
B            2    1       1
C            0    1       3

Kết quả kiểm định Chi-square cho Car:
Chi2 statistic = 5.1429
p-value = 0.2730

Bảng tần suất (Day vs Category):
 Category  High  Low  Medium
Day                        
Day 1        1    0       2
Day 2        1    0       2
Day 3        0    1       2
Day 4        0    2       1

Kết quả kiểm định Chi-square cho Day:
Chi2 statistic = 6.0952
p-value = 0.4126


In [ ]:
# Dựa trên kiểm định Chi bình phương giữa các mẫu xe và phân loại điểm số, kết quả cho thấy không có sự khác biệt giữa các mẫu A, B, và C trong suốt 4 ngày thử nghiệm (vì p = 0.273 > 0.05).
# vì cả hai p-value đều > 0.05 nên kết quả thử nghiệm không phụ thuộc vào ngày cũng không phụ thuộc vào mẫu xe 

### Bài 3:

In [7]:
import sqlite3
import pandas as pd

try:
    # Kết nối đến database
    conn = sqlite3.connect('Fflights.db')
    cursor = conn.cursor()

    # Xóa bảng nếu đã tồn tại (tránh lỗi khi chạy lại)
    cursor.execute('DROP TABLE IF EXISTS Fflights')

    # Tạo bảng Fflights
    cursor.execute('''
    CREATE TABLE Fflights (
        id INT PRIMARY KEY,
        departure_time INT
    );
    ''')

    # Thêm dữ liệu mẫu
    cursor.executemany('''
    INSERT INTO Fflights (id, departure_time) VALUES (?, ?)''', [
        (1, 830),
        (2, 1445),
        (3, 30),
        (4, 0),
        (5, 2359)
    ])

    conn.commit()

    # Hiển thị dữ liệu ban đầu
    print("\n Dữ liệu ban đầu:")
    df_before = pd.read_sql_query('SELECT * FROM Fflights', conn)
    print(df_before)

    # Thêm cột departure_time_converted
    cursor.execute('''
    ALTER TABLE Fflights ADD COLUMN departure_time_converted TEXT;
    ''')

    # Cập nhật giá trị định dạng giờ:phút (hh:mm)
    cursor.execute('''
    UPDATE Fflights
    SET departure_time_converted = 
      SUBSTR('0000' || departure_time, -4, 2) || ':' || SUBSTR('0000' || departure_time, -2, 2);
    ''')

    conn.commit()

    # Hiển thị dữ liệu sau khi cập nhật
    print("\n Dữ liệu sau khi thêm departure_time_converted:")
    df_after = pd.read_sql_query('SELECT * FROM Fflights', conn)
    print(df_after)

finally:
    # Đóng kết nối
    cursor.close()
    conn.close()



 Dữ liệu ban đầu:
   id  departure_time
0   1             830
1   2            1445
2   3              30
3   4               0
4   5            2359

 Dữ liệu sau khi thêm departure_time_converted:
   id  departure_time departure_time_converted
0   1             830                    08:30
1   2            1445                    14:45
2   3              30                    00:30
3   4               0                    00:00
4   5            2359                    23:59


### Bài 4:

In [8]:
import sqlite3
import pandas as pd

try:
    # Kết nối đến database SQLite
    conn = sqlite3.connect('Saless.db')
    cursor = conn.cursor()

    # Xóa bảng nếu đã tồn tại (tránh lỗi khi chạy lại)
    cursor.execute('DROP TABLE IF EXISTS Saless')

    # Tạo bảng Saless
    cursor.execute('''
    CREATE TABLE SAless (
        id INTEGER PRIMARY KEY,
        sale_amount INTEGER
    );
    ''')

    # Thêm dữ liệu mẫu
    cursor.executemany('''
    INSERT INTO SAless (id, sale_amount) VALUES (?, ?)''', [
        (1, 100),
        (2, 110),
        (3, 95),
        (4, 105),
        (5, 120),
        (6, 300),
        (7, 98),
        (8, 102)
    ])

    conn.commit()

    # Hiển thị dữ liệu bảng Sales
    print("\n Dữ liệu bảng Sales:")
    df_sales = pd.read_sql_query('SELECT * FROM Saless', conn)
    print(df_sales)

finally:
    # Đóng kết nối SQLite
    cursor.close()
    conn.close()



 Dữ liệu bảng Sales:
   id  sale_amount
0   1          100
1   2          110
2   3           95
3   4          105
4   5          120
5   6          300
6   7           98
7   8          102


In [10]:
import sqlite3
import pandas as pd

try:
    # Kết nối đến database SQLite
    conn = sqlite3.connect('Saless.db')
    cursor = conn.cursor()

    # Tính median bằng SQL
    query_median = '''
    SELECT AVG(sale_amount) AS median
    FROM (
      SELECT sale_amount
      FROM SAless
      ORDER BY sale_amount
      LIMIT 2 - (SELECT COUNT(*) FROM Saless) % 2
      OFFSET (SELECT (COUNT(*) - 1) / 2 FROM Saless)
    );
    '''

    # Thực thi câu truy vấn và lấy kết quả
    df_median = pd.read_sql_query(query_median, conn)
    median = df_median["median"][0]

    # Hiển thị kết quả
    print("\n Median:", median)

    conn.commit()

finally:
    # Đóng kết nối SQLite
    cursor.close()
    conn.close()



 Median: 103.5


In [15]:
import sqlite3
import pandas as pd

try:
    # Kết nối đến database SQLite
    conn = sqlite3.connect('Saless.db')
    cursor = conn.cursor()

    # Tính median (nếu chưa tính trước đó)
    query_median = '''
    SELECT AVG(sale_amount) AS median
    FROM (
      SELECT sale_amount
      FROM Saless
      ORDER BY sale_amount
      LIMIT 2 - (SELECT COUNT(*) FROM Saless) % 2
      OFFSET (SELECT (COUNT(*) - 1) / 2 FROM Saless)
    );
    '''
    df_median = pd.read_sql_query(query_median, conn)
    median = df_median["median"][0]

    # Tính MAD (Mean Absolute Deviation) bằng SQL
    query_mad = f'''
    SELECT AVG(abs_diff) AS mad
    FROM (
      SELECT ABS(sale_amount - {median}) AS abs_diff
      FROM Saless
      ORDER BY abs_diff
    );
    '''
    
    # Thực thi câu truy vấn MAD và lấy kết quả
    df_mad = pd.read_sql_query(query_mad, conn)
    mad = df_mad["mad"][0]

    # Hiển thị kết quả MAD
    print("\n MAD:", mad)

finally:
    # Đóng kết nối SQLite
    cursor.close()
    conn.close()



 MAD: 30.0


In [16]:
import sqlite3
import pandas as pd

try:
    # Kết nối đến database SQLite
    conn = sqlite3.connect('Saless.db')
    cursor = conn.cursor()

    # Tính median (nếu chưa tính trước đó)
    query_median = '''
    SELECT AVG(sale_amount) AS median
    FROM (
      SELECT sale_amount
      FROM SAless
      ORDER BY sale_amount
      LIMIT 2 - (SELECT COUNT(*) FROM Saless) % 2
      OFFSET (SELECT (COUNT(*) - 1) / 2 FROM Saless)
    );
    '''
    df_median = pd.read_sql_query(query_median, conn)
    median = df_median["median"][0]

    # Tính MAD (Mean Absolute Deviation)
    query_mad = f'''
    SELECT AVG(abs_diff) AS mad
    FROM (
      SELECT ABS(sale_amount - {median}) AS abs_diff
      FROM Saless
      ORDER BY abs_diff
    );
    '''
    df_mad = pd.read_sql_query(query_mad, conn)
    mad = df_mad["mad"][0]

    # Tìm ngoại lệ (outliers) dựa trên 1.5 * MAD
    threshold = 1.5 * mad
    query_outliers = f'''
    SELECT *
    FROM Saless
    WHERE ABS(sale_amount - {median}) > {threshold};
    '''
    
    # Lấy các ngoại lệ và hiển thị
    df_outliers = pd.read_sql_query(query_outliers, conn)
    print("\n Các giá trị ngoại lệ (dựa trên 1.5 * MAD):")
    print(df_outliers)

finally:
    # Đóng kết nối SQLite
    cursor.close()
    conn.close()



 Các giá trị ngoại lệ (dựa trên 1.5 * MAD):
   id  sale_amount
0   6          300


### Bài 5:

In [17]:
# Kết nối SQLite
conn = sqlite3.connect('patients.db')
cursor = conn.cursor()

# Tạo bảng Patient
cursor.execute('''
CREATE TABLE patients (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight INTEGER,
    height INTEGER
);
''')

# Chèn dữ liệu mẫu
cursor.executemany('''
INSERT INTO patients (id, last_name, weight, height) VALUES (?, ?, ?, ?)''', [
    (1, 'Nguyen', 60, 165),
    (2, 'Nguyen', 61, 165),
    (3, 'Tran', 55, 160),
    (4, 'Nguyen', 72, 170),
    (5, 'Tran', 55, 159),
    (6, 'Le', 65, 168)
])

# In bảng dữ liệu mẫu
print("\nDữ liệu mẫu từ bảng Patients:")
df_patients = pd.read_sql_query('SELECT * FROM patients', conn)
print(df_patients)

# Truy vấn tính điểm Boolean
query = '''
SELECT 
    p1.id AS id1, p2.id AS id2,
    p1.last_name AS name1, p2.last_name AS name2,
    p1.weight AS weight1, p2.weight AS weight2,
    
    (CASE WHEN p1.last_name = p2.last_name THEN 1 ELSE 0 END) +
    (CASE WHEN ABS(p1.weight - p2.weight) <= 2 THEN 1 ELSE 0 END) AS match_score

FROM patients p1
JOIN patients p2 ON p1.id < p2.id
WHERE
    (p1.last_name = p2.last_name) AND (ABS(p1.weight - p2.weight) <= 2);
'''

df_matches = pd.read_sql_query(query, conn)
print("\nCác cặp bệnh nhân có khả năng là cùng người:")
print(df_matches)


Dữ liệu mẫu từ bảng Patients:
   id last_name  weight  height
0   1    Nguyen      60     165
1   2    Nguyen      61     165
2   3      Tran      55     160
3   4    Nguyen      72     170
4   5      Tran      55     159
5   6        Le      65     168

Các cặp bệnh nhân có khả năng là cùng người:
   id1  id2   name1   name2  weight1  weight2  match_score
0    1    2  Nguyen  Nguyen       60       61            2
1    3    5    Tran    Tran       55       55            2
